### 2. Passing Leaders

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from datetime import date
import re

In [19]:
# CFB lines url
espn_url = 'https://www.espn.co.uk/college-football/stats/player/_/view/offense/table/passing/sort/passingYards/dir/desc'

In [20]:
# get raw data
page = requests.get(espn_url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="fittPageContainer") # found by inspecting html

In [21]:
raw_output = [i.text for i in results.find_all("td", class_="Table__TD")]

In [22]:
n_rows = 50

# get qb names
qbs = []
for i in range(n_rows*2):
    if i % 2 != 0:
        qbs.append(raw_output[i])

school = []
qb_clean = []

# split out the school from the surname of the QB
# could probably use REGEX to improve this, but it's fine
for qb in qbs:
    if qb[len(qb)-4].isupper():
        # 4 letter team abbrv
        qb_clean.append(qb[:len(qb)-4])
        school.append(qb[len(qb)-4:])
    elif qb[len(qb)-3].isupper():
        # 3 letter abbrv
        qb_clean.append(qb[:len(qb)-3])
        school.append(qb[len(qb)-3:])
    else:
        # 2 letter abbrv
        qb_clean.append(qb[:len(qb)-2])
        school.append(qb[len(qb)-2:])
        
# get other stats
raw_stats = raw_output[n_rows*2:]
table = pd.DataFrame(np.reshape(raw_stats, (int(len(raw_stats)/11), 11)), 
                     columns = ['POS','CMP','ATT','CMP%','YDS','AVG','LNG','TD','INT','SACK','RTG'])
table.insert(0, 'NAME', qb_clean)
table.insert(1, 'ABBRV', school)

In [23]:
table.head()

,NAME,ABBRV,POS,CMP,ATT,CMP%,YDS,AVG,LNG,TD,INT,SACK,RTG
0,Brennan Armstrong,UVA,QB,145,224,64.7,"1,973",8.8,65,14,4,13,155.8
1,Jake Haener,FRES,QB,136,186,73.1,"1,842",9.9,86,15,2,8,180.8
2,Will Rogers,MSST,QB,169,225,75.1,"1,454",6.5,41,11,2,7,143.7
3,Chevan Cordeiro,HAW,QB,101,185,54.6,"1,410",7.6,74,7,6,15,124.6
4,Grant Wells,MRSH,QB,96,150,64.0,"1,380",9.2,56,5,5,4,145.6


In [24]:
# save out as dataset
todays_date = str(date.today())
table.to_csv('data/'+todays_date+'-qb-records.csv', index=False)